In [1]:
import iatikit

In [2]:
from datetime import datetime
import iatikit

registry = iatikit.data()

publishers = sorted(
    [(min([d.metadata.get('metadata_created')
           for d in p.datasets]
          ), p.metadata.get('title'))
     for p in registry.publishers])

for idx, tup in enumerate(publishers):
    print('{order}: {name} ({date})'.format(
        order=(idx + 1),
        name=tup[1],
        date=datetime.strptime(tup[0], '%Y-%m-%dT%H:%M:%S.%f').date()
    ))

1: UK - Department for International Development (DFID) (2011-01-29)
2: The William and Flora Hewlett Foundation (2011-03-31)
3: The World Bank (2011-05-14)
4: Development Initiatives Poverty Research (2011-07-13)
5: Australia - Department of  Foreign Affairs and Trade (2011-09-02)
6: Netherlands - Ministry of Foreign Affairs (2011-09-19)
7: United Nations Office for Project Services (UNOPS) (2011-10-03)
8: Sweden, through Swedish International Development Cooperation Agency (Sida) (2011-11-11)
9: Oxfam GB (2011-11-25)
10: Finland - Ministry of Foreign Affairs (2011-11-25)
11: The Global Fund to Fight AIDS, Tuberculosis and Malaria (2011-11-25)
12: Asian Development Bank (2011-11-28)
13: Publish What You Fund (2011-11-29)
14: Resource Extraction Monitoring (2012-02-21)
15: ADRA-UK (2012-02-24)
16: Catholic Agency For Overseas Development (CAFOD) (2012-03-21)
17: GAVI Alliance (2012-03-30)
18: GlobalGiving.org (2012-05-09)
19: LEAD International (2012-05-24)
20: MapAction (2012-05-28)
2

833: Commune de Toffo - Bénin (2018-08-31)
834: Commune de Parakou - Bénin (2018-08-31)
835: Agence Nationale d'Approvisionnement en Eau Potable en Milieu Rural du Benin (2018-08-31)
836: Pastoralist Girls Initiative (2018-09-04)
837: United Nations High Commissioner for Refugees (UNHCR) (2018-09-04)
838: International Federation of the Red Cross and Red Crescent (IFRC) (2018-09-06)
839: Africalia (2018-09-07)
840: Stichting VU Amsterdam - Amsterdam Centre for World Food Studies (2018-09-10)
841: Netherlands Leprosy Relief (2018-09-10)
842: Global Green Growth Institute (GGGI) (2018-09-11)
843: We Effect (2018-09-12)
844: Send a Cow Kenya (2018-09-13)
845: Northern Rangelands Trust (2018-09-17)
846: Water Integrity Network Association (2018-09-19)
847: Académie de recherche et d'enseignement supérieur (2018-09-26)
848: Deloitte Touche Tohmatsu India LLP (2018-09-27)
849: CIVICUS World Alliance for Citizen Participation (2018-09-27)
850: Vlaamse Interuniversitaire Raad voor Universitair

In [37]:
import iatikit

# get the AsDB publisher
asdb = registry.publishers.find(name='asdb')

activities = asdb.activities

total_c = 0
total_d_e = 0

for activity in activities:  
    values = activity.etree.xpath('transaction[transaction-type/@code="C"]/value/text()')
    total_c += sum([float(v) for v in values])

for activity in activities:  
    values = activity.etree.xpath('transaction[transaction-type/@code="D" or transaction-type/@code="E"]/value/text()')
    total_d_e += sum([float(v) for v in values])

print(total_c)        
print(total_d_e)

156554932323.78998
92602825355.02998


In [14]:
import iatikit

registry = iatikit.data()

dfid = registry.publishers.find(name='dfid')
sector_category = iatikit.Sector(311, 2)  # Agriculture

ag_acts = dfid.activities.where(
    actual_start__lte='2017-12-31',  # started before 2018
    actual_end__gte='2017-01-01',  # ended after 2016
    sector__in=sector_category,
)
print('DFID had {:,} agricultural activities running during 2017.'.format(
    len(ag_acts)))

DFID had 201 agricultural activities running during 2017.


In [4]:
transaction_types = ['D', 'E', '3', '4']
' or '.join(['transaction-type/@code="{transaction_type}"'.format(
        transaction_type=transaction_type,
    ) for transaction_type in transaction_types])

'transaction-type/@code="D" or transaction-type/@code="E" or transaction-type/@code="3" or transaction-type/@code="4"'

In [13]:
import iatikit
from collections import defaultdict

registry = iatikit.data()

def total_transactions(transaction_name, org_name, year):
    if transaction_name == 'commitments':
        transaction_types = ['C', '2']
    elif transaction_name == 'spend':
        transaction_types = ['D', 'E', '3', '4']
    else:
        raise Exception(f'Unknown transaction name: {transaction_name}')
    org = registry.publishers.find(name=org_name)
    activities = org.activities

    total = defaultdict(float)

    type_str = ' or '.join([f'transaction-type/@code="{transaction_type}"'
                            for transaction_type in transaction_types])
    
    path = f'transaction[{type_str}]' + \
           f'[number(translate(transaction-date/@iso-date, "-", "")) >= {year}0101 ' + \
           f'and number(translate(transaction-date/@iso-date, "-", "")) < {year+1}0101]' + \
           f'/value'
    for activity in activities:
        values = activity.etree.xpath(path)
        default_currency = activity.etree.get('default-currency')
        for value in values:
            currency = value.get('currency') or default_currency
            total[currency] += float(value.text)

    print(f'Sum of {transaction_name} for {org_name} in {year}:')
    if total == {}:
        print('  No transactions of that type were found')
        out = [{
            'org_name': org_name,
            'year': year,
            'transaction_type': transaction_name,
            'currency': None,
            'value': None,
        }]
    else:
        for currency, value in total.items():
            print(f'  {currency} {value:,.0f}')
        out = [{
            'org_name': org_name,
            'year': year,
            'transaction_type': transaction_name,
            'currency': currency,
            'value': value,
        } for currency, value in total.items()]
    
    return out

In [14]:
import csv
fieldnames = ['org_name', 'year', 'transaction_type', 'currency', 'value']

org_names = [
    'asdb',
    'undp',
    'dfid',
    'afdb',
    'millenniumchallenge',
    'worldbank',
    'iadb',
    'gac-amc',
    'unicef',
    'ec-echo',
    'ec-near',
    'gavi',
    'theglobalfund',
    'sida',
    'minbuza_nl',
    'usaid',
    'ec-devco',
    'be-dgd',
    'unitedstates',
    'bmz',
    'ausgov',
    'ebrd',
    'eib',
    'finland_mfa',
    'ifcwbg',
    'sdc_ch',
    'danida',
    'bmgf',
    'france',
    'afd',
    'aics',
    'norad',
    'irishaid',
    'jica',
    'odakorea',
    'maec',
    'fco',
    'unocha',
    'mfat',
    'mofa-jpn',

]
with open('output.csv', 'w') as handler:
    writer = csv.DictWriter(handler, fieldnames=fieldnames)
    writer.writeheader()
    for org_name in org_names:
        for transaction_name in ['commitments', 'spend']:
            for year in [2017, 2016, 2015, 2014]:
                    output = total_transactions(transaction_name, org_name, year)
                    for o in output:
                        writer.writerow(o)

Sum of commitments for asdb in 2017:
  USD 17,259,534,000
Sum of commitments for asdb in 2016:
  USD 11,285,720,000
Sum of commitments for asdb in 2015:
  USD 14,626,478,567
Sum of commitments for asdb in 2014:
  USD 12,047,103,895
Sum of spend for asdb in 2017:
  USD 9,667,488,260
Sum of spend for asdb in 2016:
  USD 8,304,387,366
Sum of spend for asdb in 2015:
  USD 9,374,615,901
Sum of spend for asdb in 2014:
  USD 7,164,563,994
Sum of commitments for undp in 2017:
  No transactions of that type were found
Sum of commitments for undp in 2016:
  No transactions of that type were found
Sum of commitments for undp in 2015:
  No transactions of that type were found
Sum of commitments for undp in 2014:
  No transactions of that type were found
Sum of spend for undp in 2017:
  USD 4,172,352,111
Sum of spend for undp in 2016:
  USD 3,214,517,701
Sum of spend for undp in 2015:
  USD 2,549,181,972
Sum of spend for undp in 2014:
  USD 1,681,120,428
Sum of commitments for dfid in 2017:
  GBP 1

KeyboardInterrupt: 